In [7]:
# 🚗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')
!pip install optuna --quiet

# 📚 Librerías
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from scipy.spatial.distance import mahalanobis
import optuna
import random
random.seed(42)
np.random.seed(42)

# 📥 Cargar base funcional
ruta_base = "/content/drive/MyDrive/Datos/8_Base_Funcional_Reducida_RQ.parquet"
df = pd.read_parquet(ruta_base)

# ✅ Crear RQ_final con el valor en 2023 si existe
rq_cols = sorted([col for col in df.columns if col.startswith("RQ_") and col[3:].isdigit()],
                 key=lambda x: int(x.split("_")[1]))
df["RQ_final"] = df["RQ_2023"].where(df["RQ_2023"].notna())

# Filtrar empresas válidas
df = df[df["RQ_final"].notna()].copy()

# 📌 Separar X e y
columnas_excluir = [col for col in df.columns if col.startswith("RQ_") or col in ["DEP", "CIIU_Letra"]]
X_full = df.drop(columns=columnas_excluir)
y_full = df["RQ_final"]

# ✅ Filtrar observaciones válidas
validez_X = X_full.notna().sum(axis=1) >= 5
validez_y = y_full.notna()
validos = validez_X & validez_y

X = X_full[validos].copy()
y = y_full[validos].astype(int).copy()

# 🧼 Limpieza e imputación
X = X.replace([np.inf, -np.inf], np.nan)
for col in X.columns:
    col_vals = X[col]
    if col_vals.notna().any():
        p99 = col_vals.quantile(0.99)
        X[col] = col_vals.fillna(p99)
    else:
        X[col] = 0

# Interpolación y winsorización
X_interp = X.astype('float64').interpolate(axis=1, limit_direction='both')
q_low = X_interp.quantile(0.001)
q_high = X_interp.quantile(0.999)
X_winz = X_interp.clip(lower=q_low, upper=q_high, axis=1)

# 📉 Otras transformaciones
X_pca = pd.DataFrame(PCA(n_components=5, random_state=42).fit_transform(X_winz), index=X_winz.index)
X_derivadas = X_winz.diff(axis=1).iloc[:, 1:]
# 📏 Métricas funcionales
def L1_dist(x1, x2): return np.nansum(np.abs(x1 - x2))
def L2_dist(x1, x2): return np.sqrt(np.nansum((x1 - x2)**2))
def Prom_dist(x1, x2):
    mask = ~np.isnan(x1) & ~np.isnan(x2)
    if mask.sum() == 0: return np.inf
    return np.nansum(np.abs(x1[mask] - x2[mask])) / mask.sum()

# 📐 Mahalanobis
cov = np.cov(X_winz.dropna().T)
inv_covmat = np.linalg.pinv(cov)
def maha_dist(x1, x2): return mahalanobis(x1, x2, inv_covmat)

# ⚙️ Evaluación funcional k-NN
def evaluar_knn_funcional(X_data, y_data, distancia_func, k=15, alpha=5):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accs, precs, recs, f1s = [], [], [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_data), 1):
        print(f"\n🔁 Fold {fold} - k = {k}, alpha = {alpha:.2f}")
        X_train, X_test = X_data.iloc[train_idx], X_data.iloc[test_idx]
        y_train, y_test = y_data.iloc[train_idx], y_data.iloc[test_idx]

        preds = []
        for _, test_row in tqdm(X_test.iterrows(), total=X_test.shape[0], leave=False):
            distancias = []
            for _, train_row in X_train.iterrows():
                try:
                    d = distancia_func(test_row.values, train_row.values)
                except:
                    d = np.inf
                distancias.append((d, y_train.loc[train_row.name]))

            vecinos = sorted(distancias, key=lambda x: x[0])[:k]
            ponderacion = 0
            suma_pesos = 0

            for d, v in vecinos:
                peso = 1e6 if d == 0 else 1 / d
                if v == 1:
                    peso *= alpha
                ponderacion += peso * v
                suma_pesos += peso

            proba = ponderacion / suma_pesos if suma_pesos > 0 else 0.5
            pred = round(proba)
            preds.append(pred)

        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds, zero_division=0)
        rec = recall_score(y_test, preds, zero_division=0)
        f1 = f1_score(y_test, preds, zero_division=0)

        accs.append(acc)
        precs.append(prec)
        recs.append(rec)
        f1s.append(f1)


    return {
        "k": k,
        "alpha": alpha,
        "Accuracy": np.mean(accs),
        "Precision": np.mean(precs),
        "Recall": np.mean(recs),
        "F1-score": np.mean(f1s),
        "F1_std": np.std(f1s)
    }

# 🔍 Optuna para optimizar k y alpha en L1
def objetivo(trial):
    k = trial.suggest_int("k", 3, 25)
    alpha = trial.suggest_float("alpha", 1, 50, step=1)
    resultado = evaluar_knn_funcional(X_sample, y_sample, L1_dist, k=k, alpha=alpha)
    return resultado["F1-score"]

# 🧪 Muestra para optimización
np.random.seed(42)
muestra_idx = np.random.choice(X_winz.index, size=int(0.10 * len(X_winz)), replace=False)
X_sample = X_winz.loc[muestra_idx]
y_sample = y.loc[muestra_idx]

# 🚀 Ejecutar Optuna con semilla fija (para reproducibilidad total)
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objetivo, n_trials=30)

mejores_params = study.best_params
print("🔎 Mejores hiperparámetros encontrados:", mejores_params)
# 📊 Evaluar todas las variantes con los mejores hiperparámetros
k_opt = mejores_params["k"]
alpha_opt = mejores_params["alpha"]

resultados = []

resultados.append(
    evaluar_knn_funcional(X_winz, y, L1_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "L1"}
)
resultados.append(
    evaluar_knn_funcional(X_winz, y, L2_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "L2"}
)
resultados.append(
    evaluar_knn_funcional(X_winz, y, Prom_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "Promedio"}
)
resultados.append(
    evaluar_knn_funcional(X_pca, y, L1_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "PCA funcional"}
)
resultados.append(
    evaluar_knn_funcional(X_derivadas, y, L1_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "Derivadas funcionales"}
)
resultados.append(
    evaluar_knn_funcional(X_winz, y, maha_dist, k=k_opt, alpha=alpha_opt) | {"Métrica": "Mahalanobis"}
)

# 🧾 Convertir y guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados["Base"] = "Turismo"
df_resultados["Modelo"] = "k-NN funcional"
df_resultados["Naturaleza"] = "Funcional"
df_resultados["Temporalidad"] = "Dinámica"
df_resultados["Tipo de aprendizaje"] = "Supervisado"
df_resultados["Top 1 variable"] = ""
df_resultados["Top 2 variable"] = ""
df_resultados["Top 3 variable"] = ""

# Reordenar columnas
columnas_finales = [
    "Base", "Modelo", "Métrica", "Naturaleza", "Temporalidad", "Tipo de aprendizaje",
    "Accuracy", "Precision", "Recall", "F1-score", "F1_std", "k", "alpha",
    "Top 1 variable", "Top 2 variable", "Top 3 variable"
]
df_resultados = df_resultados[columnas_finales]

# 💾 Guardar CSV
ruta_resultados = "/content/drive/MyDrive/Resultados/17_Resultados_KNN_Funcional_Optuna.csv"
df_resultados.to_csv(ruta_resultados, index=False)
print(f"\n✅ Resultados guardados en: {ruta_resultados}")

# 👁️ Mostrar
df_resultados


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[I 2025-05-20 16:38:51,229] A new study created in memory with name: no-name-8539cd57-15b2-4f7d-a518-d969380e295f



🔁 Fold 1 - k = 11, alpha = 48.00


  0%|          | 0/37 [00:00<?, ?it/s]


🔁 Fold 2 - k = 11, alpha = 48.00


...

  0%|          | 0/365 [00:00<?, ?it/s]


🔁 Fold 5 - k = 14, alpha = 9.00


  0%|          | 0/365 [00:00<?, ?it/s]


✅ Resultados guardados en: /content/drive/MyDrive/Resultados/17_Resultados_KNN_Funcional_Optuna.csv


,Base,Modelo,Métrica,Naturaleza,Temporalidad,Tipo de aprendizaje,Accuracy,Precision,Recall,F1-score,F1_std,k,alpha,Top 1 variable,Top 2 variable,Top 3 variable
0,Turismo,k-NN funcional,L1,Funcional,Dinámica,Supervisado,0.806789,0.147520,0.375639,0.209757,0.038368,14,9.0,,,
1,Turismo,k-NN funcional,L2,Funcional,Dinámica,Supervisado,0.794753,0.129603,0.349573,0.186587,0.020607,14,9.0,,,
2,Turismo,k-NN funcional,Promedio,Funcional,Dinámica,Supervisado,0.806789,0.147520,0.375639,0.209757,0.038368,14,9.0,,,
3,Turismo,k-NN funcional,PCA funcional,Funcional,Dinámica,Supervisado,0.794747,0.130789,0.363984,0.192062,0.034932,14,9.0,,,
4,Turismo,k-NN funcional,Derivadas funcionales,Funcional,Dinámica,Supervisado,0.813344,0.150280,0.354326,0.209830,0.056931,14,9.0,,,
5,Turismo,k-NN funcional,Mahalanobis,Funcional,Dinámica,Supervisado,0.785421,0.135766,0.403611,0.202572,0.049763,14,9.0,,,
